In [28]:
# 

from eval_metrics.glossary import *

import json
def load_jsonl(path):
    data=[]
    with open(path, 'r', encoding='utf-8') as reader:
        for line in reader:
            data.append(json.loads(line))
    return data 

ROOT_PATH = "/data/aofei"
dataset = "Slake"
# lora_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/organ_brain/lora/epoch9_seed4/inference/pred.jsonl"
# bv_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/organ_brain/bbox_0.1/epoch9_seed4/inference/pred_beam.jsonl"
# gt_path = f"{ROOT_PATH}/hallucination/{dataset}/data/organ/test_brain.json"

# lora_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/organ_abd/lora/epoch9_seed4/inference/pred.jsonl"
# bv_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/organ_abd/bbox_0.1/epoch9_seed4/inference/pred_beam.jsonl"
# gt_path = f"{ROOT_PATH}/hallucination/{dataset}/data/organ/test_abd.json"

lora_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/organ_lung/lora/epoch9_seed4/inference/pred.jsonl"
bv_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/organ_lung/bbox_0.1/epoch9_seed4/inference/pred_beam.jsonl"
gt_path = f"{ROOT_PATH}/hallucination/{dataset}/data/organ/test_lung.json"

gt = json.load(open(gt_path, 'r'))
# candidate = json.load(open(args.candidate, 'r'))
lora_pred = load_jsonl(lora_pred_path)
bv_pred = load_jsonl(bv_pred_path)

gt_ids = [item['id'] for item in gt]
pred_ids = [item['question_id'] for item in lora_pred]
bv_pred_ids = [item['question_id'] for item in bv_pred]
num_gt_ids, num_pred_ids = len(gt_ids), len(pred_ids)
print(f'num_gt_ids: {num_gt_ids} || num_pred_ids: {num_pred_ids}')
# import pdb; pdb.set_trace()
assert gt_ids == pred_ids, "please make sure pred and gt are exactly matched"
assert gt_ids == bv_pred_ids, "please make sure pred and gt are exactly matched"

num_gt_ids: 419 || num_pred_ids: 419


In [29]:
# the original lora
num_close = 0
correct_ids, wrong_ids = [], []
for gt_item, pred_item in zip(gt, lora_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                correct_ids.append(_id)
            else:
                wrong_ids.append(_id)
        # else:
        #     wrong_ids.append(_id)

In [30]:
len(wrong_ids), len(correct_ids)

(13, 106)

In [31]:
# our method, without MOE
num_close = 0
bv_correct_ids, bv_wrong_ids, bv_close_fail_ids = [], [], []
for gt_item, pred_item in zip(gt, bv_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                bv_correct_ids.append(_id)
            else:
                bv_wrong_ids.append(_id)
        else:
            bv_close_fail_ids.append(_id)
        # else:
        #     nobbox_wrong_ids.append(_id)

In [32]:
len(bv_wrong_ids), len(bv_correct_ids), len(bv_close_fail_ids)

(12, 107, 0)

In [33]:
common_wrong_ids, bv_wrongs, lora_wrongs = [], [], []
for i in wrong_ids:
    if i in bv_wrong_ids:
        common_wrong_ids.append(i)
    else:
        lora_wrongs.append(i)
for i in bv_wrong_ids:
    if i in wrong_ids:
        continue
    else:
        bv_wrongs.append(i)

common_correct_ids, bv_corrects, lora_corrects = [], [], []
for i in correct_ids:
    if i in bv_correct_ids:
        common_correct_ids.append(i)
    else:
        lora_corrects.append(i)
for i in bv_correct_ids:
    if i in correct_ids:
        continue
    else:
        bv_corrects.append(i)
"correct:", len(lora_corrects), len(bv_corrects), "wrong:", len(lora_wrongs), len(bv_wrongs),

('correct:', 1, 2, 'wrong:', 2, 1)

In [34]:
gt[0]
gt_id_dict = dict()
for i in gt:
    gt_id_dict[i['id']] = i

In [35]:
# specific questions wrongly answered by our method
for i in bv_wrongs:
    print(gt_id_dict[i]["organ"], gt_id_dict[i]["conversations"])

Lung [{'from': 'human', 'value': '<image>\nDoes the picture contain liver?'}, {'from': 'gpt', 'value': 'No'}]


In [36]:
for i in lora_wrongs:
    print(gt_id_dict[i]["organ"], gt_id_dict[i]["conversations"])

Lung [{'from': 'human', 'value': '<image>\nCan atelectasis be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]
Lung [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]


In [37]:
bv_corrects
# gt_id_dict[11979]
for i in bv_corrects:
    print(gt_id_dict[i])

{'image': 'xmlab312/source.jpg', 'id': 12318, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan atelectasis be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab375/source.jpg', 'id': 12483, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}


In [38]:
wrong_after_tuning_ids = list(set(lora_corrects) and set(bv_wrongs))
print(len(wrong_after_tuning_ids), len(lora_corrects), len(bv_wrongs))

correct_after_tuning_ids = list(set(lora_wrongs) and set(bv_corrects))
len(correct_after_tuning_ids), len(lora_wrongs), len(bv_corrects)

1 1 1


(2, 2, 2)

In [39]:
set(wrong_after_tuning_ids) == set(bv_wrongs)

True

In [40]:
# wrong_after_tuning_ids
for i in wrong_after_tuning_ids:
    print(gt_id_dict[i])

{'image': 'xmlab25/source.jpg', 'id': 12164, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain liver?'}, {'from': 'gpt', 'value': 'No'}]}


In [41]:
# correct_after_tuning_ids
for i in correct_after_tuning_ids:
    print(gt_id_dict[i])

{'image': 'xmlab375/source.jpg', 'id': 12483, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab312/source.jpg', 'id': 12318, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan atelectasis be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}


s

In [ ]:
{'image': 'xmlab93/source.jpg', 'id': 12993, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain lung?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab543/source.jpg', 'id': 12802, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain the organ that could be used for secreting enzymes?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab252/source.jpg', 'id': 12195, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain kidney?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab561/source.jpg', 'id': 12836, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain colon?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab13/source.jpg', 'id': 11979, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain spleen?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab538/source.jpg', 'id': 12783, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the stomach appear in the image?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab580/source.jpg', 'id': 12880, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain heart?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab580/source.jpg', 'id': 12881, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain stomach?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab274/source.jpg', 'id': 12254, 'organ': 'Abdomen', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain spleen?'}, {'from': 'gpt', 'value': 'Yes'}]}
